In [2]:
import datetime
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from modules.Utils.utils import loadFromDB, strategyTester, getFearAndGreedIndicator,computeStochasticLinearRegression,computeEMD
from modules.Utils.indicators import computeSuperTrend, addIndicators, computeLaggingLinearRegression, computeTrixIndicator, generateDatesFeatures
from modules.Utils.normalizer import MedianNormalizer, StandardNormalizer

from random import randint
from bokeh.plotting import figure
from bokeh.io import push_notebook,show, output_notebook
from bokeh.models import HoverTool, Band, ColumnDataSource,Span
from bokeh.layouts import row, column
output_notebook()

from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocess import cpu_count

from modules.Utils.utils import computeFutureLinearRegression


2022-10-04 15:25:44.524753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-04 15:25:45.065201: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-04 15:25:45.065330: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-04 15:25:45.136201: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-04 15:25:47.516624: W tensorflow/stream_executor/platform/de

Loading BokehJS ...

In [158]:
SYMBOL= 'BTC'
df = loadFromDB(SYMBOL,'1h')
df = addIndicators(df,derivative=True, b_engulfings=True)
df = computeLaggingLinearRegression(df, 'Close',20, False,derivative=False)
df = computeStochasticLinearRegression(df,"Close")
df = computeTrixIndicator(df, col='Close')
df = generateDatesFeatures(df)
df = computeEMD(df)
#df = getFearAndGreedIndicator(df)
df.dropna(inplace=True)
print(df.shape)
print(df.columns)
df.head()

(23184, 37)
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Timestamp', 'High_Low_diff',
       'EMA20', 'EMA50', 'EMA100', 'EMA200', 'MACD', 'Stoch_RSI', 'Vortex',
       'Bollinger_low', 'Bollinger_high', 'ADX', 'ATR', 'CCI', 'OVB',
       'OVB_EMA200', 'EVM', 'Bullish_engulfing', 'Bearish_engulfing', 'Slope',
       'B_MLR_coefs', 'Stochastic_prediction', 'Trix', 'Trix_histo', 'Month',
       'Year', 'Week', 'Hour', 'Year_sin', 'Year_cos', 'EMD_S', 'EMD_D'],
      dtype='object')


,Open,High,Low,Close,Volume,Timestamp,High_Low_diff,EMA20,EMA50,EMA100,...,Trix,Trix_histo,Month,Year,Week,Hour,Year_sin,Year_cos,EMD_S,EMD_D
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-12 04:00:00,8095.00,8110.92,8065.62,8091.33,963.637569,1578798000,45.30,8089.782443,8048.379011,7976.638385,...,8102.613890,-0.052148,1,2020,2,4,0.188807,0.982014,-43.614483,8134.944483
2020-01-12 05:00:00,8090.70,8105.00,8071.20,8073.79,711.211749,1578801600,33.80,8088.259353,8049.375521,7978.562179,...,8101.261583,-0.048403,1,2020,2,5,0.189511,0.981879,-57.932028,8131.722028
2020-01-12 06:00:00,8073.03,8110.92,8070.98,8106.05,997.127263,1578805200,39.94,8089.953701,8051.598049,7981.086690,...,8099.943496,-0.041874,1,2020,2,6,0.190215,0.981743,-22.176112,8128.226112
2020-01-12 07:00:00,8105.96,8153.84,8080.01,8083.84,2509.947391,1578808800,73.83,8089.371444,8052.862440,7983.121409,...,8098.651182,-0.036312,1,2020,2,7,0.190918,0.981606,-40.603638,8124.443638
2020-01-12 08:00:00,8083.43,8110.00,8080.45,8097.51,867.095068,1578812400,29.55,8090.146544,8054.613324,7985.386530,...,8097.497336,-0.030390,1,2020,2,8,0.191622,0.981469,-22.858759,8120.368759


In [4]:
from scipy.signal import savgol_filter
INPUT_STEPS = 30
COVERAGE_STEPS = 7
OUTPUT_STEPS = 14
OUTPUT_SIZE = COVERAGE_STEPS+OUTPUT_STEPS
BATCH_SIZE = 64
LABEL_COL = 'Close_denoised'

currencies = ['BTC','ETH',]#'ADA','AAVE','ALGO','EGLD','ETH','LINK','DOT','SOL','UNI','XRP','AVAX','AXS','NEAR','MATIC']
print(f'Total currencies : {len(currencies)}')
def getCurrencies(symbol:str):
    df = loadFromDB(symbol,'12h')
    df = addIndicators(df,derivative=True, b_engulfings=True)
    df = computeLaggingLinearRegression(df, 'Close',20, False,derivative=False)
    df = computeStochasticLinearRegression(df,"Close")
    df = computeTrixIndicator(df, col='Close')
    df = generateDatesFeatures(df)
    df = getFearAndGreedIndicator(df)
    df = computeEMD(df)
    df.dropna(inplace=True)
    df.drop(columns=['Timestamp',],inplace=True)#'Close','Open','High','Low'
    df['Close_denoised'] = savgol_filter(df.Close.values, 30,2, mode='nearest')
    print(f'{symbol} : {df.shape}')
    return df.dropna()

def scaleDfs(df:pd.DataFrame,)->pd.DataFrame:
    return StandardNormalizer().fit_transform(df)


def split_sequence(sequence,
                    look_back=60,
                    forecast_horizon=20,
                    coverage=20,
                    column_labels='Close'):
    X, y = [], []
    for i in range(len(sequence)): 
        lag_end = i + look_back
        forecast_end = lag_end + forecast_horizon
        if forecast_end > len(sequence):
            break
        X.append(sequence.iloc[i:lag_end])
        y.append(sequence.iloc[lag_end-coverage:forecast_end][column_labels])
    return {'X':np.array(X),
            'y':np.array(y)}

def main(symbol:str):
    return split_sequence(scaleDfs(getCurrencies(symbol)),INPUT_STEPS,OUTPUT_STEPS,COVERAGE_STEPS, column_labels=LABEL_COL)

# dfs = []
# with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
#     processes = [executor.submit(main, cur) for cur in currencies]

# for task in as_completed(processes):
#     dfs.append(task.result())

# print('df loaded !')


# dfs_scaled=[]
# with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
#     processes = [executor.submit(scaleDfs, df_i) for df_i in dfs]

# for task in as_completed(processes):
#     dfs_scaled.append(task.result())

# print('Scaling done !')


x_n_y = []
with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
    processes = [executor.submit(main, cur) for cur in currencies]

for task in as_completed(processes):
    x_n_y.append(task.result())

print('Splitting done !')

X = np.vstack(tuple([x['X'] for x in x_n_y]))
y = np.vstack(tuple([x['y'] for x in x_n_y]))
#np.save('./data/X.npy',X)
#np.save('./data/y.npy',y)
print('Saving done !')
X.shape, y.shape

Total currencies : 2
BTC : (1423, 40)
ETH : (1423, 40)
Splitting done !


FileNotFoundError: [Errno 2] No such file or directory: './data/X.npy'

In [34]:
#INPUT_STEPS = 60
#OUTPUT_STEPS = 20
#COVERAGE_STEPS = 20
#OUTPUT_SIZE = COVERAGE_STEPS+OUTPUT_STEPS
#scaler = StandardNormalizer()
#train_df_test = scaler.fit_transform(df)
#scaler.inverse_transform(train_df_test)
#dump(scaler,f'./models/standard_normalizer_{SYMBOL}.save')
#train_df =scaler.fit_transform(df)

#def split_sequence(sequence,
#                    look_back,
#                    forecast_horizon,
#                    coverage,
#                    column_labels=['EMA20']):
#    X, y = [], []
#    for i in range(len(sequence)): 
#        lag_end = i + look_back
#        forecast_end = lag_end + forecast_horizon
#        if forecast_end > len(sequence):
#            break
#        X.append(sequence.iloc[i:lag_end])
#        y.append(sequence.iloc[lag_end-coverage:forecast_end][column_labels])
#    return np.array(X), np.array(y)


#
#X,y = split_sequence(train_df,INPUT_STEPS,OUTPUT_STEPS,COVERAGE_STEPS,column_labels=['B_MLR_coefs'])

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# LSTM

In [36]:

lstm_model = tf.keras.Sequential([
    tf.keras.layers.GRU(32, return_sequences=False,),
    tf.keras.layers.Dense(OUTPUT_SIZE*1,kernel_initializer=tf.initializers.zeros()),
    tf.keras.layers.Reshape([OUTPUT_SIZE, 1])
])

lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[tf.keras.metrics.MeanAbsoluteError()])

history = lstm_model.fit(X_train,y_train,
                    epochs=50,
                    validation_split=0.2,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        verbose=1,
                                                       patience=8,
                                                       mode='min',
                                                       restore_best_weights=True)],
                    use_multiprocessing=True)


Epoch 1/50
34/34 [==============================] - 3s 26ms/step - loss: 0.5416 - mean_absolute_error: 0.5912 - val_loss: 0.1108 - val_mean_absolute_error: 0.2671
Epoch 2/50
34/34 [==============================] - 0s 15ms/step - loss: 0.0672 - mean_absolute_error: 0.2058 - val_loss: 0.0424 - val_mean_absolute_error: 0.1535
Epoch 3/50
34/34 [==============================] - 1s 16ms/step - loss: 0.0332 - mean_absolute_error: 0.1309 - val_loss: 0.0274 - val_mean_absolute_error: 0.1145
Epoch 4/50
34/34 [==============================] - 1s 15ms/step - loss: 0.0239 - mean_absolute_error: 0.1083 - val_loss: 0.0227 - val_mean_absolute_error: 0.1023
Epoch 5/50
34/34 [==============================] - 0s 15ms/step - loss: 0.0196 - mean_absolute_error: 0.0977 - val_loss: 0.0194 - val_mean_absolute_error: 0.0952
Epoch 6/50
34/34 [==============================] - 0s 15ms/step - loss: 0.0166 - mean_absolute_error: 0.0899 - val_loss: 0.0170 - val_mean_absolute_error: 0.0890
Epoch 7/50
34/34 [====

# CONV

In [37]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.AveragePooling1D(pool_size=2, strides=1, padding='valid', data_format='channels_last'),
    tf.keras.layers.Lambda(lambda x: x[:, -3:, :]),
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(3)),
    tf.keras.layers.Dense(OUTPUT_SIZE*1,kernel_initializer=tf.initializers.zeros(),),
    tf.keras.layers.Reshape([OUTPUT_SIZE, 1])
])


conv_model.compile(loss=tf.keras.losses.MeanSquaredError(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[tf.keras.metrics.MeanAbsoluteError()])

            
history = conv_model.fit(X_train,y_train,
                    epochs=50,
                    validation_split=0.2,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        verbose=1,
                                                       patience=8,
                                                       mode='min',
                                                       restore_best_weights=True)],
                    use_multiprocessing=True)


Epoch 1/50
34/34 [==============================] - 1s 6ms/step - loss: 0.2147 - mean_absolute_error: 0.3265 - val_loss: 0.0187 - val_mean_absolute_error: 0.1020
Epoch 2/50
34/34 [==============================] - 0s 3ms/step - loss: 0.0154 - mean_absolute_error: 0.0907 - val_loss: 0.0124 - val_mean_absolute_error: 0.0756
Epoch 3/50
34/34 [==============================] - 0s 3ms/step - loss: 0.0101 - mean_absolute_error: 0.0674 - val_loss: 0.0103 - val_mean_absolute_error: 0.0674
Epoch 4/50
34/34 [==============================] - 0s 3ms/step - loss: 0.0087 - mean_absolute_error: 0.0609 - val_loss: 0.0091 - val_mean_absolute_error: 0.0632
Epoch 5/50
34/34 [==============================] - 0s 3ms/step - loss: 0.0076 - mean_absolute_error: 0.0564 - val_loss: 0.0080 - val_mean_absolute_error: 0.0589
Epoch 6/50
34/34 [==============================] - 0s 3ms/step - loss: 0.0070 - mean_absolute_error: 0.0544 - val_loss: 0.0076 - val_mean_absolute_error: 0.0569
Epoch 7/50
34/34 [==========

# Test

In [38]:
pred_lstm = lstm_model.predict(X_test)
pred_conv = conv_model.predict(X_test)

21/21 [==============================] - 0s 951us/step


In [28]:
lstm_model.save(f'./models/lstm_model_all_1h_EMA20.h5')
conv_model.save(f'./models/conv_model_all_1h_EMA20.h5')

In [40]:
ind = randint(0,len(y_test)-1)
print(ind)

p = figure(title=f"Comparaison", x_axis_label='time', y_axis_label='value ($)',width=1500, height=600,)
# Add the HoverTool to the figure
p.add_tools(HoverTool(
    tooltips=[
    ("Price", "@y{0.00} $"),
], formatters={
        '$x': 'datetime',
        '$y' : 'printf',
    },
    mode='vline'
))
p.line(list(range(len(y_test[ind]))), (pred_conv[ind].reshape(OUTPUT_SIZE)+ pred_lstm[ind].reshape(OUTPUT_SIZE))/2, legend_label="LSTM x Conv Prediction", line_width=2, color='green')
p.line(list(range(len(y_test[ind]))), pred_lstm[ind].reshape(OUTPUT_SIZE), legend_label="LSTM Prediction", line_width=2, color='orange',line_dash='dashed',alpha=.3)
p.line(list(range(len(y_test[ind]))), pred_conv[ind].reshape(OUTPUT_SIZE), legend_label="Conv Prediction", line_width=2, color='red',line_dash='dashed',alpha=.3)
p.line(list(range(len(y_test[ind]))), y_test[ind].reshape(OUTPUT_SIZE), legend_label=LABEL_COL, line_width=2, )
vline = Span(location=COVERAGE_STEPS, dimension='height', line_color='black', line_width=2,line_dash='dashed')
p.renderers.extend([vline])
p.legend.location = "top_left"

show(p)

212


# Prediction price movement

In [9]:
df['Close'].shift(-1)

Date
2020-10-01 00:00:00    10619.13
2020-10-01 12:00:00    10436.63
2020-10-02 00:00:00    10570.40
2020-10-02 12:00:00    10536.65
2020-10-03 00:00:00    10542.06
                         ...   
2022-09-25 12:00:00    18876.62
2022-09-26 00:00:00    19227.82
2022-09-26 12:00:00    20229.26
2022-09-27 00:00:00    20313.58
2022-09-27 12:00:00         NaN
Freq: 12H, Name: Close, Length: 1454, dtype: float64

In [30]:
df['Close'].rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0]).tail(10)

Date
2022-09-23 00:00:00    0.0
2022-09-23 12:00:00    1.0
2022-09-24 00:00:00    0.0
2022-09-24 12:00:00    0.0
2022-09-25 00:00:00    1.0
2022-09-25 12:00:00    0.0
2022-09-26 00:00:00    1.0
2022-09-26 12:00:00    1.0
2022-09-27 00:00:00    1.0
2022-09-27 12:00:00    1.0
Freq: 12H, Name: Close, dtype: float64

In [31]:
df['Close'].tail(10)

Date
2022-09-23 00:00:00    18862.72
2022-09-23 12:00:00    19289.91
2022-09-24 00:00:00    19051.63
2022-09-24 12:00:00    18920.50
2022-09-25 00:00:00    19117.94
2022-09-25 12:00:00    18807.38
2022-09-26 00:00:00    18876.62
2022-09-26 12:00:00    19227.82
2022-09-27 00:00:00    20229.26
2022-09-27 12:00:00    20313.58
Freq: 12H, Name: Close, dtype: float64

In [61]:
def getCurrencies(symbol:str):
    df = loadFromDB(symbol,'12h')
    df = addIndicators(df,derivative=True, b_engulfings=True)
    #df = computeLaggingLinearRegression(df, 'Close',20, False,derivative=False)
    #df = computeStochasticLinearRegression(df,"Close")
    df = computeTrixIndicator(df, col='Close')
    df = generateDatesFeatures(df)
    df = getFearAndGreedIndicator(df)
    df = computeEMD(df)
    df.dropna(inplace=True)
    df.drop(columns=['Timestamp',],inplace=True)#'Close','Open','High','Low'
    print(f'{symbol} : {df.shape}')
    df['Direction'] = df['Close'].rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0])
    return df.dropna()

df = getCurrencies('BTC')
SHIFT=1

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
X = StandardScaler().fit_transform(df.iloc[:-SHIFT].values)
y = df['Direction'].iloc[SHIFT:].values
X_train, X_test, y_train, y_test =  train_test_split(X,y,shuffle=True,test_size=0.5,)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier()#RandomForestClassifier(n_estimators=70, max_depth=5,)

clf.fit(X_train, y_train)
clf.score(X_test, y_test)


for threshold in range(30,70,2):
    df_res = pd.DataFrame(clf.predict_proba(X_test),columns=['Down','Up'])
    df_res['True_y'] = y_test
    df_res['Pred_y'] = df_res.apply(lambda x:1.0 if x['Up'] > threshold/100 else 0.0,axis=1)
    df_res['Validation'] = df_res.apply(lambda x:1.0 if x['True_y'] == x['Pred_y'] else 0.0,axis=1)
    print('Threshold : ',threshold/100, 'Result : ',df_res['Validation'].sum()/len(df_res))


BTC : (1454, 37)
Threshold :  0.3 Result :  0.5909090909090909
Threshold :  0.32 Result :  0.5977961432506887
Threshold :  0.34 Result :  0.6129476584022039
Threshold :  0.36 Result :  0.6129476584022039
Threshold :  0.38 Result :  0.6033057851239669
Threshold :  0.4 Result :  0.5977961432506887
Threshold :  0.42 Result :  0.6046831955922864
Threshold :  0.44 Result :  0.6046831955922864
Threshold :  0.46 Result :  0.6046831955922864
Threshold :  0.48 Result :  0.6115702479338843
Threshold :  0.5 Result :  0.6033057851239669
Threshold :  0.52 Result :  0.6005509641873278
Threshold :  0.54 Result :  0.5950413223140496
Threshold :  0.56 Result :  0.5909090909090909
Threshold :  0.58 Result :  0.5785123966942148
Threshold :  0.6 Result :  0.5661157024793388
Threshold :  0.62 Result :  0.5495867768595041
Threshold :  0.64 Result :  0.5440771349862259
Threshold :  0.66 Result :  0.5440771349862259
Threshold :  0.68 Result :  0.5247933884297521


# Tuning

In [ ]:
#import keras_tuner
#
#def build_model(hp):
#    CONV_WIDTH = hp.Int("CONV_WIDTH", min_value=2, max_value=6, step=2)
#    model = tf.keras.Sequential([
#        # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
#        tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
#        # Shape => [batch, 1, conv_units]
#        tf.keras.layers.Conv1D(256, activation=hp.Choice("activation", ["relu", "tanh"]), kernel_size=(CONV_WIDTH)),
#        # Shape => [batch, 1,  out_steps*features]
#        tf.keras.layers.Dense(OUTPUT_SIZE*1,
#                            kernel_initializer=tf.initializers.zeros()),
#        # Shape => [batch, out_steps, features]
#        tf.keras.layers.Reshape([OUTPUT_SIZE, 1])
#    ])
#
#    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                        patience=8,
#                                                        mode='min')
#
#    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
#    model.compile(loss=tf.keras.losses.MeanSquaredError(),
#                optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
#                metrics=[tf.keras.metrics.MeanAbsoluteError()])
#    return model
#
#
##build_model(keras_tuner.HyperParameters())
#
#tuner = keras_tuner.RandomSearch(
#    hypermodel=build_model,
#    objective="val_mean_absolute_error",
#    max_trials=10,
#    executions_per_trial=1,
#    overwrite=True,
#    #directory="my_dir",
#    #project_name="helloworld",
#)
#tuner.search_space_summary()
#
#
#tuner.search(X,
#            y,
#            epochs=30,
#            validation_split=0.2,
#            #callbacks=[early_stopping],
#            use_multiprocessing=True)
#            
#models = tuner.get_best_models(num_models=2)
#model = models[0]

In [59]:
class TFLiteModel:
    """Wrapper class for loading and using a tflite model."""
    def __init__(self, model_path: str)-> None:
        """Constructor : load tflite model.

        Args:
            model_path (str): The path to the model.
        """
        self._interpreter = tf.lite.Interpreter(model_path=model_path)
        self._interpreter.allocate_tensors()
        
    def predict(self, input_data:np.ndarray,unscale_data:bool=False) -> np.ndarray:   
             
        input_details = self._interpreter.get_input_details()
        output_details = self._interpreter.get_output_details()
        assert input_data.shape == tuple(input_details[0]['shape'])
        
        self._interpreter.set_tensor(input_details[0]['index'], np.float32(input_data))
        self._interpreter.invoke()
        output_data = self._interpreter.get_tensor(output_details[0]['index'])
        return output_data

In [72]:
class TFModel:
    """Wrapper class for loading and using a tflite model."""
    def __init__(self, model_path: str)-> None:
        """Constructor : load tflite model.

        Args:
            model_path (str): The path to the model.
        """
        #self._interpreter = Interpreter(model_path=model_path)
        self._model = tf.keras.models.load_model(model_path)

        
    def predict(self, input_data:np.ndarray,unscale_data:bool=False) -> np.ndarray:   
        assert input_data.shape[1] == self._model.input_shape[1]
        assert input_data.shape[2] == self._model.input_shape[2]
        output_data = self._model.predict(input_data)
        return output_data

lstm_model_loaded = TFModel(f'./Models/BTC_lstm_model.h5')
lstm_model_loaded.predict(np.float32(X[9683].reshape(1,60,38)))

1/1 [==============================] - 0s 312ms/step


array([[[ 1.6916391e-02],
        [ 5.9132818e-03],
        [-4.5452788e-03],
        [-1.1907976e-02],
        [-1.1918915e-02],
        [ 7.2451495e-04],
        [ 2.7115535e-02],
        [ 7.0385814e-02],
        [ 1.3892455e-01],
        [ 2.2964500e-01],
        [ 3.4193736e-01],
        [ 4.7728604e-01],
        [ 6.2671155e-01],
        [ 7.7918184e-01],
        [ 9.2151755e-01],
        [ 1.0459557e+00],
        [ 1.1578847e+00],
        [ 1.2516888e+00],
        [ 1.3165712e+00],
        [ 1.3439410e+00],
        [ 1.3241885e+00],
        [ 1.2188883e+00],
        [ 1.0429329e+00],
        [ 8.1587952e-01],
        [ 5.3970832e-01],
        [ 2.2437508e-01],
        [-1.2387420e-01],
        [-4.9179712e-01],
        [-8.6253297e-01],
        [-1.2226083e+00],
        [-1.5615782e+00],
        [-1.8622689e+00],
        [-2.1124802e+00],
        [-2.2995503e+00],
        [-2.4196582e+00],
        [-2.4709282e+00],
        [-2.4529319e+00],
        [-2.3751953e+00],
        [-2.

In [60]:
conv_model=tf.keras.models.load_model(f'./models/{SYMBOL}_conv_model.h5')
lstm_model=tf.keras.models.load_model(f'./models/{SYMBOL}_lstm_model.h5')
predictions = []
# Convert the model
for model,name in zip([conv_model,lstm_model],['conv','lstm']):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.allow_custom_ops = True
    converter.target_spec.supported_ops = [
      tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
      tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
    ]
    tflite_model = converter.convert()
    # Save the model.
    with open(f'./models/{SYMBOL}_{name}.tflite', 'wb') as f:
      f.write(tflite_model)

    current_model = TFLiteModel(f'./models/{SYMBOL}_{name}.tflite')
    predictions.append(current_model.predict(np.float32(X[9683].reshape(1,60,38))).reshape(40))


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpfrkj20xo\assets


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpfrkj20xo\assets


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpm8wjg3ll\assets


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpm8wjg3ll\assets


In [62]:
p = figure(title=f"Comparaison", x_axis_label='time', y_axis_label='value ($)',width=1500, height=600,)
# Add the HoverTool to the figure
p.add_tools(HoverTool(
    tooltips=[
    ("Price", "@y{0.00} $"),
], formatters={
        '$x': 'datetime',
        '$y' : 'printf',
    },
    mode='vline'
))
p.line(list(range(len(predictions[1]))), (predictions[1].reshape(OUTPUT_SIZE)+ predictions[0].reshape(OUTPUT_SIZE))/2, legend_label="LSTM x Conv Prediction", line_width=2, color='orange')
p.line(list(range(len(predictions[1]))), y[9683].reshape(OUTPUT_SIZE), legend_label="B_MLR_coefs", line_width=2, )
vline = Span(location=COVERAGE_STEPS, dimension='height', line_color='black', line_width=2,line_dash='dashed')
p.renderers.extend([vline])
p.legend.location = "top_left"
show(p)

In [ ]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="./model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.float32(X)#np.array(np.float32(X[-1].reshape(1,60,38)), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'],np.float32(X[-1].reshape(1,60,38)))
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

In [139]:
df.B_MLR_coefs.values


array([ 2.81817043e-04,  1.80724728e-04, -9.02046784e-05, ...,
        6.11131997e-05, -2.60066834e-05, -8.37984545e-04])

In [140]:
output_data[0].reshape(40)*df.B_MLR_coefs.std()+df.B_MLR_coefs.mean()

array([-0.00382623, -0.00524257, -0.00597959, -0.00604487, -0.00516733,
       -0.00501163, -0.00449779, -0.00386011, -0.00400485, -0.00375702,
       -0.00344385, -0.00304197, -0.00313159, -0.00264799, -0.00356175,
       -0.00303906, -0.00250716,  0.00064374,  0.00176276,  0.00520646,
        0.00802915,  0.01000458,  0.00997781,  0.01054154,  0.01024669,
        0.00866046,  0.00766866,  0.0082461 ,  0.00879483,  0.0097452 ,
        0.01036937,  0.00972864,  0.00815944,  0.00612878,  0.00232412,
       -0.00146018, -0.00498969, -0.00818477, -0.01031332, -0.01007655],
      dtype=float32)

In [144]:
np.vstack([df.B_MLR_coefs.values
,output_data[0].reshape(40)*df.B_MLR_coefs.std()+df.B_MLR_coefs.mean()])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 23184 and the array at index 1 has size 40